In [ ]:
import os
import re
from string import digits

from bs4 import BeautifulSoup as BS
import json

import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz, process

In [ ]:
def clear_soup(xml_doc):
    dic = {'<lb></lb>': ' ', '<lb/>': ' ', '¬ ': '', '\n' : ' '}
    for i, j in dic.items():
        xml_doc = xml_doc.replace(i, j)
    xml_doc = re.sub('<fw type="breaking-entry"/>', ' ', xml_doc)
    xml_doc = re.sub('<fw.*?</fw>', ' ', xml_doc)
    xml_doc = re.sub(' +', ' ', xml_doc)
    return xml_doc

In [ ]:
all_bibl = []


def find_bibl(entry):
    bibl = entry.find_all('bibl')
    s_lst, t_lst, bibl_saved = [], [], []
    if bibl:
        for i in bibl:
            bibl_saved.append(str(i)) 
            if '</hi>' in i:
                i.hi.decompose
            source = i.find_all('title', {"class": "source"})
            if source:
                for s in source:
                    s_lst.append(s.text)
            text = i.find_all('title', {"class": "text"})
            if text:
                for t in text:
                    t_lst.append(t.text) 
    return s_lst, t_lst, bibl_saved


for volume in range(1, 28):
    f = 'C:/Users/Tuylenen/Desktop/лексикография/xml_printed_edition/oldrus-tei-' + str(volume) + '.xml'
    with open(f, 'r', encoding='utf-8') as xml_doc:
    
        xml_doc = xml_doc.read()
        xml_doc = clear_soup(xml_doc)
        
        soup = BS(xml_doc, 'lxml')
        
        for entry in soup.find_all("entry", {'type' : ['mainEntry', 'xref']}):
            entry_saved = str(entry)
            entry_data = re.search('type="(.*?)" xml:id=["\'](.*?)["\']', str(entry))
            if entry_data:
                main_id = entry_data.group(2)
                if 'id_' in main_id:
                    main_id = re.sub('id_[0-9]*_', '', main_id)
                type_entry = entry_data.group(1)
                s_lst, t_lst, bibl_saved = find_bibl(entry)
            
                all_bibl.append([volume, main_id, main_id, type_entry, s_lst, t_lst, bibl_saved, entry_saved])
            else:
                print('NOTHING', entry)
            sub_entries = entry.find_all("entry")
            if sub_entries:
                for sub_entry in sub_entries:
                    entry_data = re.search('type="relatedEntry" xml:id=["\'](.*?)["\']', str(sub_entry))
                    entry_id = None
                    if entry_data:
                        entry_id = entry_data.group(1)    
                    s_lst, t_lst, bibl_saved = find_bibl(sub_entry)
                    all_bibl.append([volume, main_id, entry_id, 'reatedEntry', s_lst, t_lst, bibl_saved, entry_saved])

NOTHING <entry type="xref" xml:lang="orv"> <form type="lemma"><orth>ΚЫ-</orth></form> <xr><lbl type="see">см.</lbl> <ref type="lemma">ки-</ref></xr> </entry>


In [ ]:
df = pd.DataFrame(all_bibl, columns=['volume', 'main_entry', 'entry', 'type', 'source', 'text', 'bibl', 'entry'])  

In [ ]:
df

,volume,main_entry,entry,type,source,text,bibl,entry
0,1,А1,А1,mainEntry,"[Ж. Феодос. Нест., Смол. гр.]",[(Дог. смол. кн. с Ригою)],"[<bibl><title class=""source"">Ж. Феодос. Нест.<...","<entry type=""mainEntry"" xml:id=""А1"" xml:lang=""..."
1,1,А2,А2,mainEntry,"[РИБ, Ав. Ж., Сл. о п. Иг., Х. Тр. Короб., Х. ...",[(Дан. киев. кн.)],"[<bibl><title class=""text"">(Дан. киев. кн.)</t...","<entry type=""mainEntry"" xml:id=""А2"" xml:lang=""..."
2,1,А2,А_любо,reatedEntry,[Лавр. лет.],[],"[<bibl><title class=""source"">Лавр. лет.</title...","<entry type=""mainEntry"" xml:id=""А2"" xml:lang=""..."
3,1,А2,А_то,reatedEntry,[Ав. Кн. толк.],[],"[<bibl><title class=""source"">Ав. Кн. толк.</ti...","<entry type=""mainEntry"" xml:id=""А2"" xml:lang=""..."
4,1,А2,Аже,reatedEntry,[],[],[],"<entry type=""mainEntry"" xml:id=""А2"" xml:lang=""..."
...,...,...,...,...,...,...,...,...
90767,27,СТАРИНЦЫ,СТАРИНЦЫ,mainEntry,[Заб. Мат.],[],"[<bibl><title class=""source"">Заб. Мат.</title>...","<entry type=""mainEntry"" xml:id=""СТАРИНЦЫ"" xml:..."
90768,27,СТАРИТИСЯ,СТАРИТИСЯ,mainEntry,[Сим. Послов.],[],"[<bibl><title class=""source"">Сим. Послов.</tit...","<entry type=""mainEntry"" xml:id=""СТАРИТИСЯ"" xml..."
90769,27,СТАРИЦА,СТАРИЦА,mainEntry,"[Патерик Син., Ж. Андр. Юрод.1, Флавий. Полон....",[(Лаод. 11)],"[<bibl><title class=""source"">Патерик Син.</tit...","<entry type=""mainEntry"" xml:id=""СТАРИЦА"" xml:l..."
90770,27,СТАРИЦА,Старицы_черные_,reatedEntry,"[АМГ, Гр. Крут. еп.]",[],"[<bibl><title class=""source"">АМГ</title> <bibl...","<entry type=""mainEntry"" xml:id=""СТАРИЦА"" xml:l..."


In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/all_sources_info.txt',
           df.values, fmt='%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s',
           header="'выпуск', 'статья', 'слово', 'тип', 'источник', 'текст', 'bibl', 'tei статьи'", 
           encoding="utf-8")

In [ ]:
all_sources = []

for n in df.source:
    for i in n:
        i = re.sub(' +', ' ', i)
        all_sources.append(i)

with open("C:/Users/Tuylenen/Desktop/лексикография/unique_sources.txt", "w", encoding='utf-8') as source_file:
    for source in sorted(set(all_sources)):
        source_file.write("%s\n" % source)

In [ ]:
vals = df.source.values.tolist()
rs = [len(r) for r in vals]    
a = np.repeat(df.main_entry, rs)

df_individual = pd.DataFrame(np.column_stack((a, np.concatenate(vals))), columns=["main_entry", "source"])

In [ ]:
df_individual 

,main_entry,source
0,А1,Ж. Феодос. Нест.
1,А1,Смол. гр.
2,А2,РИБ
3,А2,Ав. Ж.
4,А2,Сл. о п. Иг.
...,...,...
222484,СТАРИЦА,АЮБ
222485,СТАРИЦА,Дух. и дог. гр.
222486,СТАРИЦА,АМГ
222487,СТАРИЦА,Гр. Крут. еп.


In [ ]:
individual_source_to_entry = []

for i in df_individual.source.unique():
    
    all_entr = df_individual.loc[df_individual.source == i]
    individual_source_to_entry.append([i, all_entr.main_entry.unique().tolist()])

In [ ]:
individual_source_to_entry

[['Ж. Феодос. Нест.',
  ['А1',
   'АЛКАНИЕ',
   'БДѢТИ',
   'БѢДИТИ1',
   'БЕСЧИСЛЕННЫЙ',
   'БЛАГОДАТСТВО',
   'БЛАГОСЛОВЕСТИТИ',
   'БЛАГОЧЕСТИВЫЙ',
   'БОГОУГОДНЫЙ',
   'БОДРОСТЬ',
   'БОЛЕСТЬ',
   'БОЛЯРОВЪ',
   'БЪЧИ',
   'БРАНИТИ',
   'БРЕМЯ',
   'БУБЕНЪ',
   'ВГРЫЗЕНИЕ',
   'ВДАВАТИ',
   'ВЕЛЬЗАВЕЛИНЪ',
   'ВѢНЕЦЪ',
   'ВЕЧЕРЯ',
   'ВЗВОЛОЧИ',
   'ВЗВОРОТИТИСЯ',
   'ВЗЫСКАНИЕ',
   'ВЗЫТИЕ',
   'ВИКИЯ',
   'ВЛАСЯНЫЙ',
   'ВМЕТАТИ',
   'ВНИМАНИЕ',
   'ВОЗВЕЛИЧАТИ',
   'ВОЗВѢЩЕНИЕ',
   'ВОЗВРАТИТИСЯ',
   'ВОЗГРАЖЕНИЕ',
   'ВОЗЛЕЖАТИ',
   'ВОЗЛОЖИТИ',
   'ВОЗРАСТИ',
   'ВОСКРАЙ',
   'ВОСПОМЯНОВЕНИЕ',
   'ВПУСТИТИ',
   'ВРАТАРЬ',
   'ВСЕЛИТИ',
   'ВЫЗЫВАТИ',
   'ВЯЩИЙ',
   'ИЗГОНИТИСЯ',
   'ИСПОВѢДАНИЕ',
   'ИСТЯЗАТИ',
   'ЛИШЕ']],
 ['Смол. гр.',
  ['А1',
   'АЖЕ',
   'БЕРЕГЪ',
   'БОГОРОДИЦА',
   'ВѢСЕЦЪ',
   'ВѢСИТИ',
   'ВЕСТИ1',
   'ВЗАЕМЪ',
   'ВИНОВАТЫЙ',
   'ВОЗВРѢЩИ',
   'ВОЛОСТЕЛЬ',
   'ВОЛЬНЫЙ',
   'ВОРОТИТИ1',
   'ВОСКЛАДЫВАТИ',
   'ВОЩЕЦЪ',
   'ВСАДИТИ',
   'ВЫДАТИ',
   '

In [ ]:
df_ind = pd.DataFrame(individual_source_to_entry, columns=['source', 'main_entry']) 
df_ind

,source,main_entry
0,Ж. Феодос. Нест.,"[А1, АЛКАНИЕ, БДѢТИ, БѢДИТИ1, БЕСЧИСЛЕННЫЙ, БЛ..."
1,Смол. гр.,"[А1, АЖЕ, БЕРЕГЪ, БОГОРОДИЦА, ВѢСЕЦЪ, ВѢСИТИ, ..."
2,РИБ,"[А2, АБЫ, АВГУСТНЫЙ, АЛТАРНИКЪ, АЛЬФА, АНАГНОС..."
3,Ав. Ж.,"[А2, АДЪ, АДОВЪ, АМОЖЕ, БАБА2, БАЗЛУКЪ, БАЛАГА..."
4,Сл. о п. Иг.,"[А2, АБЫ, АКИ, АКСАМИТЪ, БАГРЯНЫЙ, БДѢТИ, БЕБР..."
...,...,...
6222,Ж.Ал.Чел.Бож.,[СТАРѢЙШИЙ]
6223,Хроногр. XV в.,[СТАРѢИШИНА]
6224,Зап.(Г.),[СТАРѢЙШИНСТВО]
6225,Кн. прих.-расх. Волок. м.,[СТАРЕЦЪ]


In [ ]:
df_ind['freq'] = df_ind.main_entry.str.len()

In [ ]:
df_ind = df_ind.sort_values(by=['source'])

In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/sources_freq.txt', df_ind.values, fmt='%s\t%s\t%s', encoding="utf-8")

In [ ]:
all_title = []
all_title_in_bibl = []

all_biblScope = []
all_biblScope_in_bibl = []

 
for volume in range(1, 28):
    f = 'C:/Users/Tuylenen/Desktop/лексикография/xml_printed_edition/oldrus-tei-' + str(volume) + '.xml'
    with open(f, 'r', encoding='utf-8') as xml_doc:
        
        xml_doc = xml_doc.read()
        xml_doc = re.sub('\n', ' ', xml_doc)
        soup = BS(xml_doc, 'lxml')
        
        for entry in soup.find_all("entry"):
                for t1 in entry.find_all("title"):
                    all_title.append((volume, entry, t1))
                #for b1 in entry.find_all("biblscope", {'unit' : 'page'}):
                for b1 in entry.find_all("biblscope"):
                    all_biblScope.append((volume, entry, b1))
            
                bibl = entry.find_all('bibl')
                if bibl:
                    for i in bibl:
                        for t in i.find_all("title"):
                            all_title_in_bibl.append((volume, entry, t))
                        #for b in i.find_all("biblscope", {'unit' : 'page}):
                        for b in i.find_all("biblscope"):
                            all_biblScope_in_bibl.append((volume, entry, b))

In [ ]:
s = set(all_title_in_bibl)

In [ ]:
temp3 = [x for x in all_title if x not in s]

In [ ]:
len(temp3)

122

In [ ]:
s2 = set(all_biblScope_in_bibl)
temp2 = [x for x in all_biblScope if x not in s2]

In [ ]:
len(temp2)

163

In [ ]:
title_df = pd.DataFrame(temp3, columns = ['volume', 'entry', 'title'])
title_df

,volume,entry,title
0,3,"[ , [[ВОТКНУТИСЯ]], (, [[ВЪТЪКНУТИСЯ]], ), [[...",[Панд. Ант.]
1,3,"[ , [[ВСЕДѢТЕЛЬНЫЙ]], , , [[прил.]], , [к , [...",[Панд. Ант.]
2,8,"[ , [[ЛАКНУТИ]], и , [[ЛОКНУТИ]], , , [[однок...",[Ст. о см.]
3,9,"[ , [[МЕДЪ]], , , [[м.]], , [1. , [Мед...",[Арханг. лет.]
4,9,"[ , [Медъ сытити, ста¬, [], вити, варити], . ...",[Арханг. лет.]
...,...,...,...
117,27,"[ , [[СРЕДОКРЕСТИЕ]], , , [[СРЕДОХРЕСТИЕ]], , ...",[ВМЧ]
118,27,"[ , [[СРУДИТИСЯ]], , [[]], . , [ , [Испа...",[Сл. Шрове]
119,27,"[ , [[СРЯЧА]], или , [[СРЯЧЬ]], , , [[ж.]], ...",[Ряз. корм.]
120,27,"[ , [[ССѢДАТИСЯ]], , [[]], . , [1. ...",[Ипат. лет.]


In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/title_out.txt', title_df.values, fmt='%s\t%s\t%s', encoding="utf-8")

In [ ]:
page_df = pd.DataFrame(temp2, columns = ['volume', 'entry', 'page'])
page_df

In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/page_out.txt', page_df.values, fmt='%s\t%s\t%s', encoding="utf-8")

In [ ]:
f = 'C:/Users/Tuylenen/Desktop/лексикография/unique_sources.txt'
with open(f, 'r', encoding='utf-8') as xml_doc:
    sources_our = xml_doc.read().split('\n')

In [ ]:
f = 'C:/Users/Tuylenen/Desktop/лексикография/sources_official.txt'
with open(f, 'r', encoding='utf-8') as xml_doc:
    sources_official = xml_doc.read().split('\n')

In [ ]:
sources_official

In [ ]:
def match_lists(list1, list2):
    matches = []
    probabilities = []

    for i in list1:
        ratios = process.extractOne(i, list2)

        if int(ratios[1]) > 93:
            matches.append(ratios[0])
            probabilities.append(ratios[1])
        else:
            matches.append('no match')
            probabilities.append('probability was too low')

    df = pd.DataFrame({
        'originals': list1,
        'potential matches': matches,
        'probability of match': probabilities
    }

    )

    return df

In [ ]:
matches = match_lists(sources_our, sources_official)

In [ ]:
sources_our_red = []

for n in sources_our:
    if n.startswith('('):
        n = n.strip('()')
    #pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    #n = re.sub(pattern, '', n)
    n = re.sub('\n+', ' ', n)
    n = re.sub(' +', ' ', n)
    n = re.sub('\. +?', '.', n)
    n = re.sub('[\-\–\—] +?', '–', n)
    n = re.sub('\.$', '', n)
    n = n.lower()
    sources_our_red.append(n.strip(' '))

In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/match.txt', matches.values, fmt='%s\t%s\t%s', encoding="utf-8")

In [ ]:
f = 'C:/Users/Tuylenen/Desktop/лексикография/match.txt'
sources_match = pd.read_csv(f, sep='\t',names = ['unq', 'right', 'prob'], engine='python')

In [ ]:
sources_match

,unq,right,prob
0,NaN,no match,probability was too low
1,Ав. Сотв. мира,Ав.Сотв.мира,95
2,Ворон. Петр. а.,Ворон.Петр.а.,95
3,Истом. Град. ц.,Истом.Град ц. Карион,95
4,Кир. Тур.,Кир.Тур. XV,95
...,...,...,...
6223,публ. б. л.,no match,probability was too low
6224,сп. Моск. дух. ак.,no match,probability was too low
6225,т.ж.,no match,probability was too low
6226,там же,no match,probability was too low


In [ ]:
match_to_right = {}

for i in sources_match.right.unique():
    all_entr = sources_match.loc[sources_match.right == i]
    match_to_right[i] = all_entr.unq.tolist()

In [ ]:
jsonStr = json.dumps(match_to_right)
json.loads(jsonStr)

{'no match': [nan,
  'Русской грамматике” Лудольфа',
  '(1 Кор. XIV, 35)',
  '(Io. Dam. Dial., PG, t. 94. col. 624)',
  '(SJS 34, 614)',
  '(Ак.)',
  '(Амф.)',
  '(Апост. XIV в.)',
  '(БАН2)',
  '(В. I, 363)',
  '(В. II, 39)',
  '(В. II. 131)',
  '(В.)',
  '(В.).',
  '(Варc. крм.)',
  '(Варс. крм.)',
  '(Волог. сб. Ак. н. XV в.)',
  '(Др. Рос. гос. II, 107)',
  '(Евг.)',
  '(Златостр. XII в.)',
  '(Изв. Арх. т. II. 109)',
  '(Изв. т. VIII, 76)',
  '(Ист. гос. Рос., т. VI, пр. 310)',
  '(Ист. чт. II. 153)',
  '(Исх. XXII, 13)',
  '(Калайд. 198)',
  '(Кар. И. Г. Р. X. 324 пр.)',
  '(Кар. И. Г. Р. т. IX. пр. 648)',
  '(Кар. И. Г. Р. т. VII, пр. 103)',
  '(Кар. И. Г. Р. т. X. пр. 426)',
  '(Кар. И. Г. Р., т. IX, пр. 648)',
  '(Кар. И.Г.Р. VI пр. 103)',
  '(Карамз. ИГР IX, прим. 348)',
  '(Кол. II, 8)',
  '(Конст. 1—2 кан. 6)',
  '(Кор. 1. И. 1)',
  '(Лев. XXIII. 34)',
  '(Лих.)',
  '(М.)',
  '(Мат. 21)',
  '(Мат. 48)',
  '(Мат. Бусл. 55)',
  '(Мин. июн. д. 1200 г.)',
  '(Н.-Кес. 4)',
  '(Н

In [ ]:
with open("matched.json", "w") as outfile:
    outfile.write(jsonStr)

In [ ]:
matches_red = match_lists(sources_our_red, sources_official)

In [ ]:
np.savetxt(r'C:/Users/Tuylenen/Desktop/лексикография/match_redacted.txt', matches_red.values, fmt='%s\t%s\t%s', encoding="utf-8")

In [ ]:
official_matched = []

for i in match_to_right.keys():
    official_matched.append(i)

In [ ]:
diff = set(sources_official) - set(official_matched)
len(diff)

2125

2125 vs 1957 (168) - разница между мэтчингом источников из указателя и источников без предобработки и с ней

In [ ]:
with open("C:/Users/Tuylenen/Desktop/лексикография/not_matched.txt", "w", encoding='utf-8') as f:
    for i in sorted(diff):
        f.write("%s\n" % i)